Hi everyone !<br/>
For this Kernel I want to make a tutorial about Text-mining with Spark. What's Spark ? What's Text-mining ? Why Spark could be interesting in Kaggle challenge ? And why Spark and Text-mining are good together ? Are the questions I will try to anwser.
<br/>
I will not make data visualization in this tutorial, I'm pretty sure a lot of people are far better than me for visualization. Unfortunatelly this notebook will not run on Kaggle because you need to install Spark.
<br/>
<h1> What's Apache Spark </h1>
Spark ( https://spark.apache.org/ ) is a framework for distributed operations (basically the opposite of using one core on one computer) it works with several language : Java,Scala,R and Python. You use it with Hadoop when you have a too many data for one computer. So why create a tutorial for this challenge with only few data ? Well, when I look at all big challenge on Kaggle I'm pretty sure that some people do feature engineering and even ML with Spark in order to gain some time, and I really think that Spark will be more and more popular for ML and Kaggle challenge and I want to help people to begin with Spark.<br/>
<br/>
First you need to know that generelly Spark run with Hadoop on several computer but you can easly install it on your own computer (there are a lot of tutorial on internet) but remember the main purpose of Spark is to run on several computer.<br/>
<br/>
<h4> MapReduce operation </h4>
Spark use the programming model mapreduce. A MapReduce program is composed of a Map() procedure (method) and a Reduce() procedure (method). In our case you (for data manipulation on dataset) Map is an operation that apply a function on each row of your dataset (example : replace the id by 1 on every line) and Reduce an operation that apply a function for combining your row (example : sum all the id of our data).<br/>
<br/>
In this notebook I would only use map so basically when you read :<br/>
example_2=example_1.map(lambda x : normalize_token(x))<br/>
It mean "apply function normalize_token on every row of dataset example_1 this new dataset name is example_2"<br/>
<br/>
So if I have 1 computer with 4 core the function will apply on 4 lines simultaneously, if I have 5 computer with 4 core on each the function will apply on 20 lines simultaneously !<br/>
<br/>
<h4> RDD VS Dataframe </h4>
Since the beginning I'm speaking about dataset but it's not really correct, you need to know they are two different way to store your data in spark, both are distributed (your data are on several computer ).<br/>
Basically Dataframe are similar to Pandas dataframe, with column, row a lot of function that already exist and are fast(example : droping NA) but you can't store every type and do what you want (a line with more element than an other etc..)<br/>
On the other side you have RDD (Resilient Distributed Dataset) basically it's just a big array and every element(row) is a tuple(similar to an array), you can create your own function with MapReduce, all element(row) can have different size, and no one care about the type of your object in your RDD.<br/>
You can easly switch RDD to DF and more easly DF to RDD, we will use both because both have advantages.<br/>
<br/>
<h1> Text-mining 101 </h1>
<br/>
In order to apply a Machine-learning algorithms you need a vector of number, so we need to transform our text into a vector. Classique methods that do this transformation need a BoW (bag of words) basically an array with word that help to classify the sentence.<br/>
But before tranforming our text into vector we need to clean it. <br/>
<br/>
<h4> Cleaning your text </h4>
Indeed there are some useless words like "the", "I","of" etc... they are use too often or they doesn't mean precise things, they are call "stopwords". <br/>
It's not the only transformation we can do, it's also important to change word to their lemma, basically change 'is','are'... to "be" and change "rows" to "row", this way word with the same meaning would write the same way.<br/>
<br/>
We can do more transformation like replace some word/acronym by other but I think you get the idea : you need to keep/extract information and reduce noise of your data.<br/>
<br/>
All those transformation exist with NLTK (natural language toolkit) a free library for text-mining in python<br/>
<br/>
After all this transformation you get an array of word (or BoW) now you can transform it into a vector of word<br/>
<br/>
<h4> BoW to vector </h4>
<br/>
There are two popular method for this, word2vec and tf-idf. I will not explain them they are a lot of explaination on internet and english is not my main language. But you juste need to know that :<br/>
- W2V use the way word are used in order to change word into vectors and have similar vector for word that are used in a similar way<br/>
- tf-idf mean term frequency inverse document frequency, for each phrase it create a vector of the size of your vocabulary and give them an important weight if they appear few time.<br/>
<br/>
Both methods can be good (like every machine learning algorithms)<br/>
Once you have change your BoW to a vector you just need to apply a RandomForest or other and it's done.<br/>
<br/>
<h1> Why Spark is good with text-mining </h1>
By reading this tutorial you probably realise that we transform a lot our data for text-mining challenge, and all those transformation are easy to parallelize. Indeed it's just applying a function on each row (so a map). So if you start a Text-mining challenge with billion of line, Spark would be really faster than using one core.<br/>
<br/>
After explaining the basics lets have a look at the code

In [ ]:
import nltk
import numpy as np
from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
sc

First let's load our training data

In [ ]:
df=spark.read.csv("train.csv",header=True)
df.show()

In this function we create a Bag of word with our sentence, and count the number of char (it could be an interessting feature)

In [ ]:
def normalize_token(x) :
    tokenizer = TreebankWordTokenizer()
    lower=x.text.lower()
    text_token=tokenizer.tokenize(lower)
    count_char=len(lower)
    return x+(count_char,text_token,)

rdd_normalize_token=df.rdd.map(lambda x : normalize_token(x))
rdd_normalize_token.first()

In this function we remove stopword, little word and punctuation, we count the number of character that are punctuation and replace all number by "number" (I want to keep the information that an author can use a lot of number) in order to normalize them

In [ ]:
def remove_stop_word_and_changer_number(x,seuil,stop_words,list_punct) :
    tab_result=[]
    count_punct=0
    for elt in x[4] :
        if elt in list_punct :
            count_punct=count_punct+1
        try :
            number=float(elt)
            tab_result.append("number")
        except :
            if (len(elt)>seuil) &(elt not in stop_words) :
                tab_result.append(elt)            
    return x[:4]+(count_punct,tab_result,)
    
seuil=1
stop_words=set(stopwords.words('english'))
list_punct=list(string.punctuation)
rdd_stop_word=rdd_normalize_token.map(lambda x : remove_stop_word_and_changer_number(x,seuil,stop_words,list_punct))
rdd_stop_word.first()

We lemmatize our BoW with NLTK, we use POSTagging to help lemmatization

In [ ]:
def lemmatisation(x,dict_cor) :
    tab_result=[]
    wordnet_lemmatizer = WordNetLemmatizer()
    pos_tmp = nltk.pos_tag(x[5])
    for elt in pos_tmp :
        if elt[1][0] in dict_cor :
            attrib=dict_cor[elt[1][0]]
        else :
            attrib = "n"
        tab_result.append(wordnet_lemmatizer.lemmatize(elt[0], pos=attrib))
    return x[:5]+(tab_result,)
        
dict_cor={
    "N" : "n",
    "V" : "v",
    "J" : "r",
    "A" : "a",
}

rdd_lemma=rdd_stop_word.map(lambda x : lemmatisation(x,dict_cor))
rdd_lemma.first()

We calcul the number of word in our BoW (it could be an interessting feature)

In [ ]:
def size_word(x) :
    nb_word=len(x[5])
    return x+(nb_word,)

rdd_size=rdd_lemma.map(lambda x :size_word(x))
rdd_size.first()

After all our modification we transform our RDD to a dataframe

In [ ]:
df_final=rdd_size.toDF(["id","phrase","Author","nb_carac","nb_punct","words","size"])
print df_final.count()
df_final.show()

Now we can save our DF to parquet (a distributed format, more efficient than csv)

In [ ]:
df_final.write.parquet("tokenize_03_12_v3",mode="overwrite")

All those previous transformation where on training set, let's do the same on test set

In [ ]:
df_test=spark.read.csv("test.csv",header=True)
 #The map is important in order to have the same number of column as train set
df_test_1=df_test.rdd.map(lambda x : x+(1,)).toDF(["id","text","author"])
df_test_2=df_test_1.rdd.map(lambda x : normalize_token(x))
df_test_3=df_test_2.map(lambda x : remove_stop_word_and_changer_number(x,seuil,stop_words,list_punct))
df_test_4=df_test_3.map(lambda x : lemmatisation(x,dict_cor))
df_test_5=df_test_4.map(lambda x :size_word(x))
df_final_test=df_test_5.toDF(["id","phrase","Author","nb_carac","nb_punct","words","size"]).drop("Author")
print df_final_test.count()
print df_final_test.show()
df_final_test.write.parquet("tokenize_test_03_12_v3",mode="overwrite")

Now we can start Machine-learning

In [ ]:
import nltk
import numpy as np
from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import Word2Vec
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator,  RegressionEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.linalg import Vectors, VectorUDT

We load our data

In [ ]:
df = spark.read.parquet("tokenize_03_12_v3")
df_test = spark.read.parquet("tokenize_test_03_12_v3")
df.show()

Let's create different categorie for each author, we would then create one model for each author

In [ ]:
def add_label(x) :
    if x.Author=="EAP" :
        EAP=1
        HPL=0
        MWS=0
    elif x.Author=="HPL" :
        EAP=0
        HPL=1
        MWS=0
    elif x.Author=="MWS" :
        EAP=0
        HPL=0
        MWS=1
    else :
        EAP=0
        HPL=0
        MWS=0
    return x+(EAP,HPL,MWS)

rdd_label=df.rdd.map(lambda x : add_label(x))
df_add_label=rdd_label.toDF(["id","phrase","Author","nb_carac","nb_punct","words","size","label_EAP","label_HPL","label_MWS"])
df_add_label.show()

They are several way to transform a BoW to an vector, tf-idf and word2Vec are pretty popular, I get better result whith w2v (train on test corpus and train corpus). But you can try with tf-idf, w2v or even both (concatenation is just after)

In [ ]:
method="both"

if method=="tf-idf" :
    hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=256)
    featurizedData = hashingTF.transform(df_add_label)

    idf = IDF(inputCol="rawFeatures", outputCol="features")
    idfModel = idf.fit(featurizedData)
    rescaledData = idfModel.transform(featurizedData)

elif method=="w2v" :
    word2Vec = Word2Vec(vectorSize=100, minCount=1, inputCol="words", outputCol="features",seed=42, maxIter=20)
    df_all_words=df_test.select("words").union(df_add_label.select("words"))
    model_w2v = word2Vec.fit(df_all_words)

    rescaledData = model_w2v.transform(df_add_label)
    
elif method=="both" :
    hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=256)
    featurizedData = hashingTF.transform(df_add_label)

    idf = IDF(inputCol="rawFeatures", outputCol="features_tf_idf")
    idfModel = idf.fit(featurizedData)
    rescaledData_tmp = idfModel.transform(featurizedData)
    
    word2Vec = Word2Vec(vectorSize=100, minCount=1, inputCol="words", outputCol="features",seed=42, maxIter=20)
    df_all_words=df_test.select("words").union(df_add_label.select("words"))
    model_w2v = word2Vec.fit(df_all_words)

    rescaledData = model_w2v.transform(rescaledData_tmp)
    
rescaledData.show()

Concatenate all feature (from w2v and/or tf-idf and external caracter) in order to train a model

In [ ]:
if method=="both" :
    df_ML=rescaledData.rdd.map(lambda x : x[:-1]+(Vectors.dense(list(x.features)+list(x.features_tf_idf) + [x.size,x.nb_carac,x.nb_punct]),))\
    .toDF(rescaledData.columns)
else :
    df_ML=rescaledData.rdd.map(lambda x : x[:-1]+(Vectors.dense(list(x.features) + [x.size,x.nb_carac,x.nb_punct]),))\
    .toDF(rescaledData.columns)
df_ML.first()

Now we train a model for each author, with cross validation and parameter tuning

In [ ]:
trainingData=rescaledData
#print trainingData.first()

nb_eap = RandomForestRegressor(featuresCol="features", predictionCol="EAP",labelCol="label_EAP", maxDepth=10, numTrees=20)

paramGrid = ParamGridBuilder() \
    .addGrid(nb_eap.numTrees, [ 40,50,60]) \
    .addGrid(nb_eap.maxDepth, [ 5,10]) \
    .build()

crossval = CrossValidator(estimator=nb_eap,
                          estimatorParamMaps=paramGrid,
                          evaluator= RegressionEvaluator(predictionCol="EAP",labelCol="label_EAP"),
                          numFolds=3)

model_eap=crossval.fit(trainingData)

In [ ]:
trainingData=rescaledData
#print trainingData.first()

nb_hpl = RandomForestRegressor(featuresCol="features",labelCol="label_HPL", predictionCol="HPL", maxDepth=10, numTrees=20)

paramGrid = ParamGridBuilder() \
    .addGrid(nb_hpl.numTrees, [40,50,60]) \
    .addGrid(nb_hpl.maxDepth, [ 5,10]) \
    .build()

crossval = CrossValidator(estimator=nb_hpl,
                          estimatorParamMaps=paramGrid,
                          evaluator= RegressionEvaluator(labelCol="label_HPL", predictionCol="HPL"),
                          numFolds=3)

model_hpl=crossval.fit(trainingData)

In [ ]:
trainingData=rescaledData
#print trainingData.first()

nb_mws = RandomForestRegressor(featuresCol="features",labelCol="label_MWS", predictionCol="MWS", maxDepth=10, numTrees=20)

paramGrid = ParamGridBuilder() \
    .addGrid(nb_mws.numTrees, [40,50,60]) \
    .addGrid(nb_mws.maxDepth,  [5,10]) \
    .build()

crossval = CrossValidator(estimator=nb_mws,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(labelCol="label_MWS", predictionCol="MWS"),
                          numFolds=3)

model_mws=crossval.fit(trainingData)

In [ ]:
print model_eap.bestModel 
print model_hpl.bestModel 
print model_mws.bestModel 

Now we apply our model to the test set

In [ ]:
df_test.show()
if method=="tf-idf" :
    featurizedData_test = hashingTF.transform(df_test)
    listfeaturized=featurizedData_test.collect()
    rescaledData_test = idfModel.transform(featurizedData_test)
    listidfMode=rescaledData_test.collect()
    featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(rescaledData_test)
    df_to_predict=featureIndexer.transform(rescaledData_test)
     df_to_predict=df_to_predict_tmp.rdd.map(lambda x : x[:-1]+(Vectors.dense(list(x.features_tf_idf) + [x.size,x.nb_carac,x.nb_punct]),))\
    .toDF(df_to_predict_tmp.columns)

elif method=="w2v" :
    df_to_predict_tmp= model_w2v.transform(df_test)
    df_to_predict=df_to_predict_tmp.rdd.map(lambda x : x[:-1]+(Vectors.dense(list(x.features) + [x.size,x.nb_carac,x.nb_punct]),))\
    .toDF(df_to_predict_tmp.columns)
    print df_to_predict.first()
    
elif method=="both" :
    list0=df_test.collect()
    featurizedData_test = hashingTF.transform(df_test)
    rescaledData_test = idfModel.transform(featurizedData_test)
    featureIndexer = VectorIndexer(inputCol="features_tf_idf", outputCol="indexedFeatures").fit(rescaledData_test)
    df_to_predict=featureIndexer.transform(rescaledData_test)
    df_to_predict_tmp= model_w2v.transform(df_to_predict)
    df_to_predict=df_to_predict_tmp.rdd.map(lambda x : x[:-1]+(Vectors.dense(list(x.features)+list(x.features_tf_idf) + [x.size,x.nb_carac,x.nb_punct]),))\
    .toDF(df_to_predict_tmp.columns)
    print df_to_predict.first()

df_test_1 = model_eap.transform(df_to_predict).drop('rawPrediction').drop('probability')
df_test_2 = model_hpl.transform(df_test_1).drop('rawPrediction').drop('probability')
df_test_3 = model_mws.transform(df_test_2).drop('phrase').drop('words').drop('rawFeatures').drop('rawPrediction').drop('probability').drop('features').drop('indexedFeatures')
print df_test_3.show()

Just to be sure that we haven't huge value, then select only column for Kaggle

In [ ]:
def change_val(x) :
    EAP=x.EAP
    HPL=x.HPL
    MWS=x.MWS
    if x.EAP>1 :
        EAP=1
    if x.HPL>1 :
        HPL=1
    if x.MWS>1 :
        MWS=1
    if x.EAP<0 :
        EAP=0
    if x.HPL<0 :
        HPL=0
    if x.MWS<0 :
        MWS=0
    return (x.id,EAP,HPL,MWS)

df_save=df_test_3.rdd.map(lambda x : change_val(x)).toDF(['id','EAP','HPL','MWS'])

Now we reduce the distribution of the DF to 1 (in ordre to have one csv) then we save it !

In [ ]:
df_save=df_test_3.coalesce(1)
print df_save.count()
print df_save.first()

In [ ]:
df_save.select(['id','EAP','HPL','MWS']).write.csv("result_test_03_12_v4",sep=",",header=True,mode="overwrite")

Well I hope this notebook will help you for your text-mining project, I also hope you learn things and want to try Spark, If you have question I will try to anwser them. Thanks for reading !